> First time use: follow instructions in the README.md file in this directory.


**[PT]** Português

---

**[EN]** English


# Toponímia


Identificação e geolocalização dos topónimos

---

# Place names

Identification and geocoding of place names


## Setup

In [1]:
from timelinknb import get_db

db_spec =  ('sqlite','fauc.db')
db = get_db(db_spec)

## Lista de lugares diferentes e número de ocorrências

---

## List of different places with number of occurrences

In [2]:
from timelinknb.pandas import attribute_values

attribute = 'naturalidade'
period = ('1500-00-00','1990-00-00')

places = attribute_values(attribute,dates_between=period)
places['place_name'] = places.index.values
places.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11499 entries, Lisboa to Óvoa, Viseu
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   count       11499 non-null  int64 
 1   date_min    11499 non-null  object
 2   date_max    11499 non-null  object
 3   place_name  11499 non-null  object
dtypes: int64(1), object(3)
memory usage: 449.2+ KB


### Lugares principais

---

### Main locations

In [3]:
places.sort_values('count', ascending=False).head(10)



,count,date_min,date_max,place_name
value,,,,
Lisboa,8784,1537-02-12,1916-07-19,Lisboa
Coimbra,5526,1537-00-00,1915-10-12,Coimbra
Porto,3391,1537-05-30,1917-10-22,Porto
Braga,1608,1540-01-21,1914-07-24,Braga
Évora,1072,1537-11-22,1910-10-10,Évora
Viseu,986,1537-00-00,1912-07-03,Viseu
Guimarães,980,1537-12-18,1912-07-18,Guimarães
Lamego,972,1537-00-00,1909-10-05,Lamego
Aveiro,790,1538-04-21,1913-10-13,Aveiro


### Lugares só com uma ocorrência
---

### Locations with just one occurrence

In [4]:
places[places['count'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Index: 7554 entries, - Lisboa to Óvoa, Viseu
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   count       7554 non-null   int64 
 1   date_min    7554 non-null   object
 2   date_max    7554 non-null   object
 3   place_name  7554 non-null   object
dtypes: int64(1), object(3)
memory usage: 295.1+ KB


In [5]:
places[places['count']==1].head(10)

,count,date_min,date_max,place_name
value,,,,
- Lisboa,1,1658-10-02,1658-10-02,- Lisboa
- Vila Franca,1,1658-10-01,1658-10-01,- Vila Franca
-Lisboa,1,1723-10-01,1723-10-01,-Lisboa
"A de Barros, Caria",1,1765-10-01,1765-10-01,"A de Barros, Caria"
"A de Barros, Lamego",1,1624-10-10,1624-10-10,"A de Barros, Lamego"
A dos Francos,1,1745-10-01,1745-10-01,A dos Francos
"ALgaça, Poiares",1,1751-10-01,1751-10-01,"ALgaça, Poiares"
AZagães,1,1749-10-01,1749-10-01,AZagães
Abade,1,1747-12-14,1747-12-14,Abade


# Identificação de topónimos

---

# Geocoding

https://craftingdh.netlify.app/tutorials/folium/

## Toponímia de Portugal Continental 1:200 000

 ### Serviço Nacional de Informação Geográfica

* Metadados https://snig.dgterritorio.gov.pt/rndg/srv/por/catalog.search#/metadata/57479cf3-df10-47a0-9860-f7e3157596b1
* Dados: http://mapas.dgterritorio.pt/ATOM-download/SCN200k/toponimia/toponimia.zip
* Acesso público sem restrições
* Sempre que o utilizador publique e/ou divulgue, por meio analógico ou digital, informação geográfica propriedade da Direção-Geral do Território, ainda que parcialmente adaptada, deverá atribuir créditos com inclusão do texto "Informação geográfica cedida pela Direção-Geral do Território"

Notas: 
* Inclui mapas e informação de layer formato dbf.
* Tabela de dados não inclui coordenadas mas inclui um id  (gml_id) que pode ser usado para obter as coordenadas noutras bases.
* Pode ser útil para corrigir problemas de ortografia nos topónimos da base de dados

### Convert informação para Pandas DataFrame

In [6]:
!pip install simpledbf

  Using cached simpledbf-0.2.6.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for simpledbf, since package 'wheel' is not installed.
  Running setup.py install for simpledbf ... done


In [7]:
import pandas as pd
from simpledbf import Dbf5

file_name = '../extras/geocoding/SNIG/portugal-continental-200k/Toponimia200k.dbf'
dbf = Dbf5(file_name)
print("Number of records:",dbf.numrec)
dbf.fields

PyTables is not installed. No support for HDF output.
Number of records: 7054


[('DeletionFlag', 'C', 1),
 ('gml_id', 'C', 254),
 ('beginLifes', 'C', 20),
 ('localId', 'C', 15),
 ('namespace', 'C', 35),
 ('versionId', 'N', 10),
 ('Integer', 'N', 10),
 ('mostDetail', 'N', 10),
 ('language', 'C', 3),
 ('sourceOfNa', 'C', 3),
 ('pronunciat', 'C', 254),
 ('text', 'C', 49),
 ('script', 'C', 4),
 ('relatedSpa', 'C', 27),
 ('relatedS_1', 'C', 45),
 ('relatedS_2', 'N', 10)]

These are included in the INE data. But do not overlap. 

In [8]:
topo200K_df = dbf.to_dataframe()
topo200K_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7054 entries, 0 to 7053
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   gml_id      7054 non-null   object 
 1   beginLifes  7054 non-null   object 
 2   localId     0 non-null      float64
 3   namespace   7054 non-null   object 
 4   versionId   7054 non-null   int64  
 5   Integer     7054 non-null   int64  
 6   mostDetail  7054 non-null   int64  
 7   language    7054 non-null   object 
 8   sourceOfNa  7054 non-null   object 
 9   pronunciat  0 non-null      float64
 10  text        7054 non-null   object 
 11  script      7054 non-null   object 
 12  relatedSpa  7054 non-null   object 
 13  relatedS_1  0 non-null      float64
 14  relatedS_2  0 non-null      float64
dtypes: float64(4), int64(3), object(8)
memory usage: 826.8+ KB


In [9]:
topo200K_df.head()

,gml_id,beginLifes,localId,namespace,versionId,Integer,mostDetail,language,sourceOfNa,pronunciat,text,script,relatedSpa,relatedS_1,relatedS_2
0,PT.GN.112300,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Praia de Faro Nascente,Latn,CDG200k_populatedPlace0,NaN,NaN
1,PT.GN.112301,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Belmonte Baixo,Latn,CDG200k_populatedPlace1,NaN,NaN
2,PT.GN.112302,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Sagres,Latn,CDG200k_populatedPlace2,NaN,NaN
3,PT.GN.112303,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Faro,Latn,CDG200k_populatedPlace3,NaN,NaN
4,PT.GN.112304,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Rio Seco,Latn,CDG200k_populatedPlace4,NaN,NaN


In [126]:
topo200K_df[topo200K_df.gml_id == 'PT.GN.50']

,gml_id,beginLifes,localId,namespace,versionId,Integer,mostDetail,language,sourceOfNa,pronunciat,text,script,relatedSpa,relatedS_1,relatedS_2,inferredType


Não vemos uma coluna a identificar o tipo de topónimo
mas é possível inferir a partir do campo "relatedSpa"
que tem um código único com um prefixo

In [122]:
topo200K_df['inferredType'] = topo200K_df.relatedSpa.str.strip("CDG_k0123456789")
topo200K_df.groupby(['inferredType'])['inferredType'].count()

inferredType
buildingForte          28
buildingSantuario      14
landformCabo           10
landformIlha           19
landformPonta          58
landformSerra         159
populatedPlace       6766
Name: inferredType, dtype: int64

In [125]:
topo200K_df[topo200K_df.inferredType.str.contains('Santuario')]

,gml_id,beginLifes,localId,namespace,versionId,Integer,mostDetail,language,sourceOfNa,pronunciat,text,script,relatedSpa,relatedS_1,relatedS_2,inferredType
6680,PT.GN.118980,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Fátima,Latn,CDG200k_buildingSantuario28,NaN,NaN,buildingSantuario
6681,PT.GN.118981,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Imaculado Coração de Maria,Latn,CDG200k_buildingSantuario29,NaN,NaN,buildingSantuario
6682,PT.GN.118982,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,N. Srª da Conceição,Latn,CDG200k_buildingSantuario30,NaN,NaN,buildingSantuario
6683,PT.GN.118983,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,N. Srª dos Milagres,Latn,CDG200k_buildingSantuario31,NaN,NaN,buildingSantuario
6684,PT.GN.118984,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,S. Bartolumeu,Latn,CDG200k_buildingSantuario32,NaN,NaN,buildingSantuario
6685,PT.GN.118985,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Srª de Nazo,Latn,CDG200k_buildingSantuario33,NaN,NaN,buildingSantuario
6686,PT.GN.118986,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,N. Srª dos Anúncios,Latn,CDG200k_buildingSantuario34,NaN,NaN,buildingSantuario
6687,PT.GN.118987,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Cristo Rei,Latn,CDG200k_buildingSantuario35,NaN,NaN,buildingSantuario
6688,PT.GN.118988,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Senhora da Rocha,Latn,CDG200k_buildingSantuario36,NaN,NaN,buildingSantuario
6689,PT.GN.118989,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Srº da Pedra,Latn,CDG200k_buildingSantuario37,NaN,NaN,buildingSantuario


In [86]:
topo200K_df.head(10)

,gml_id,beginLifes,localId,namespace,versionId,Integer,mostDetail,language,sourceOfNa,pronunciat,text,script,relatedSpa,relatedS_1,relatedS_2,inferredTtype
0,PT.GN.112300,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Praia de Faro Nascente,Latn,CDG200k_populatedPlace0,NaN,NaN,CDG200k_populatedPlace
1,PT.GN.112301,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Belmonte Baixo,Latn,CDG200k_populatedPlace1,NaN,NaN,CDG200k_populatedPlace
2,PT.GN.112302,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Sagres,Latn,CDG200k_populatedPlace2,NaN,NaN,CDG200k_populatedPlace
3,PT.GN.112303,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Faro,Latn,CDG200k_populatedPlace3,NaN,NaN,CDG200k_populatedPlace
4,PT.GN.112304,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Rio Seco,Latn,CDG200k_populatedPlace4,NaN,NaN,CDG200k_populatedPlace
5,PT.GN.112305,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Montenegro,Latn,CDG200k_populatedPlace5,NaN,NaN,CDG200k_populatedPlace
6,PT.GN.112306,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Quinta do Eucalipto,Latn,CDG200k_populatedPlace6,NaN,NaN,CDG200k_populatedPlace
7,PT.GN.112307,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Virgilios,Latn,CDG200k_populatedPlace7,NaN,NaN,CDG200k_populatedPlace
8,PT.GN.112308,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Cova da Onça,Latn,CDG200k_populatedPlace8,NaN,NaN,CDG200k_populatedPlace
9,PT.GN.112309,2015-01-01T00:00:00Z,NaN,http://id.igeo.pt/so/GN/NamedPlaced,2015,577791,144447,Por,DGT,NaN,Pontes de Marchil,Latn,CDG200k_populatedPlace9,NaN,NaN,CDG200k_populatedPlace


In [101]:
topo200k_file = '../inferences/places/topo_200k.csv'
topo200K_df[['gml_id','text','sourceOfNa']].to_csv(topo200k_file,index=False)

### Cruzar com topónimos da base local

In [27]:
!pip install recordlinkage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.5/926.5 kB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.7/28.7 MB 13.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 8.1 MB/s eta 0:00:00a 0:00:01


In [78]:
import recordlinkage
from recordlinkage.preprocessing import clean

indexer = recordlinkage.index.SortedNeighbourhood('place_name','text',window=11)
candidates = indexer.index(places,topo200K_df)
print(len(candidates))

52284


In [67]:
compare = recordlinkage.Compare()
compare.string('place_name','text',
    # ['jaro', 'jarowinkler', 'levenshtein', 'damerau_levenshtein', 'qgram', 'cosine', 'smith_waterman', 'lcs'].
    method='damerau_levenshtein', 
    threshold=0.90,
    label='score')
features = compare.compute(candidates,places,topo200K_df)
features.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 52284 entries, ('Guimarães', 2792) to ('Óbidos, Brasil', 6679)
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   score   52284 non-null  float64
dtypes: float64(1)
memory usage: 1.2+ MB


### Total de topónimos identificados

In [68]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0     3524
0.0    48760
dtype: int64

#### Verificar inferências

In [71]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()

potential_matches['topo']=topo200K_df.loc[potential_matches['level_1']]['text'].values
potential_matches['topo_id']=topo200K_df.loc[potential_matches['level_1']]['gml_id'].values
potential_matches[potential_matches.value != potential_matches.topo].head(50)

,value,level_1,score,topo,topo_id
0,Marmeleira,3106,1.0,Marmeleiro,PT.GN.115406
1,Souto Covo,4944,1.0,Souto Novo,PT.GN.117244
2,Souto Covo,6484,1.0,Souto Novo,PT.GN.118784
3,Escalas de Baixo,2187,1.0,Escalos de Baixo,PT.GN.114487
4,Aldeia Nossa do Cabo,2494,1.0,Aldeia Nova do Cabo,PT.GN.114794
5,Carvalhos,4039,1.0,Carvalhosa,PT.GN.116339
6,Carvalhos,5352,1.0,Carvalhosa,PT.GN.117652
7,Reguengo de Monsaraz,270,1.0,Reguengos de Monsaraz,PT.GN.112570
8,Santa Marinha de Zêzere,4941,1.0,Santa Marinha do Zêzere,PT.GN.117241
9,Carvalheira,4489,1.0,Carvalheiro,PT.GN.116789


Check how many not found we can match

In [72]:
from os.path import exists

not_found_file = '../inferences/places/osm_not_found.csv'

not_found_df: pd.DataFrame = None

if exists(not_found_file):
    not_found_df = pd.read_csv(not_found_file)
    not_found = list(not_found_df['not_found'])
else:
    not_found = []
    not_found_df = pd.DataFrame(columns=['not_found'])


In [73]:
nfs = not_found_df['not_found'].values
not_found_but_in_topo = potential_matches[potential_matches['value'].isin(nfs)].sort_values(['value','topo'])
not_found_but_in_topo[not_found_but_in_topo['value'] != not_found_but_in_topo['topo']]

,value,level_1,score,topo,topo_id
82,Aldeia de Joane,2905,1.0,Aldeia de Joanes,PT.GN.115205
88,Alhos Vedras,768,1.0,Alhos Vedros,PT.GN.113068
3453,Arcos de Valedevez,6900,1.0,Arcos de Valdevez,PT.GN.119200
76,Avelãs de Caminha,3101,1.0,Avelãs de Caminho,PT.GN.115401
3457,Avelãs do Caminho,3101,1.0,Avelãs de Caminho,PT.GN.115401
3464,Cabeceiras de Bastos,5617,1.0,Cabeceiras de Basto,PT.GN.117917
3465,Celorico de Bastos,5155,1.0,Celorico de Basto,PT.GN.117455
3,Escalas de Baixo,2187,1.0,Escalos de Baixo,PT.GN.114487
3514,Figueró dos Vinhos,2222,1.0,Figueiró dos Vinhos,PT.GN.114522
3459,Freixiandas,2088,1.0,Freixianda,PT.GN.114388
